# Preamble

In [5]:
import math
import numpy as np
from functools import reduce
from scipy.optimize import fsolve
from typing import Callable

# Step 1: Preprocessing + Amplitude Encoding

We need to encode our feature set with a shape that will work with a quantum circuit architecture.

## Preprocessing

Map to higher dimension by padding up to the next power of 2, padding with garbage.

In [6]:
def next_pow_of_2(n: int) -> int:
    return 2 ** (int(math.log2(n)) + 1)

def padding_size(n: int) -> int:
    return next_pow_of_2(n) - n

## Upscaling

Per some hyperparameter `d`, map to `d`-many repetitions of the feature state. The padding is applied to
the new `d`-repeated feature state.
This is just one pass through. We need to do this for every input.

In [7]:
def repeat_flat(feature_vec: list[float], repeat_times: int) -> np.ndarray:
    feature_size = len(feature_vec)
    return np.repeat(
        np.array(feature_vec)
          .reshape(1, feature_size),
        repeat_times,
        axis=0
    ).reshape(feature_size * repeat_times)

def preprocess(input_vec: list[float], repeat_times: int) -> np.ndarray:
    repeated_input = repeat_flat(input_vec, repeat_times)
    # TODO: OR we want not all 0's here! YES NEEDS TO BE NON-ZEROS!!!
    features = repeated_input + ([0] * padding_size(len(repeated_input)))
    norm = np.linalg.norm(features)
    return features / norm


## Amplitude Encoding

To put our classical data into the quantum circuit, we leverage amplitude encoding.

In [20]:
#TODO: we don't want this anymore, drop it when sure we don't need any of this logic elsewhere
#      ALSO drop any unneeded imports from preamble
'''
def column_tensor_product(lhs: np.ndarray, rhs: np.ndarray) -> np.ndarray:
    print('lhs col tensor', lhs)
    print('rhs col tensor', rhs)
    return np.tensordot(lhs, rhs, axes=0) \
             .reshape(len(lhs) * len(rhs))

def generate_state_tensor(states: list[np.ndarray]) -> np.ndarray:
    return reduce(
        lambda accum, curr: column_tensor_product(accum, curr),
        states
    )

def tensor_product_equations(flattened_state_vecs: list[float], feature_values: list[float]) -> list[float]:
    state_vecs = np.array(flattened_state_vecs) \
        .reshape(int(len(flattened_state_vecs) / 2), 2)
    print('TP equalities. state_vecs', state_vecs)
    print('TP equalities. feature_values', feature_values)
    state_tensor = generate_state_tensor(state_vecs)
    
    print('TP equalities. state_tensor', state_tensor)
    # each component of the state tensor needs to be equal to that component from the feature vector
    # and each qubit state vector must have α^2 + β^2 = 1
    eqs = list(state_tensor - feature_values) #\
        #+ [np.sum([q_i ** 2 for q_i in state]) - 1 for state in state_vecs]
    print(eqs)
    return eqs

#def create_tensor_product_equations(feature_values: list[float]) -> Callable[[list[np.ndarray]], list[float]]:
#    return lambda state_vecs: tensor_product_equalities(state_vecs, feature_values)

def compute_initial_states(qubit_count: int, feature_vec: list[float]) -> list[list[float]]:
    zeroes = [np.zeros(2) for i in range(qubit_count)]
    states = fsolve(tensor_product_equations, zeroes, args=feature_vec)
    return states


compute_initial_states(3, [0,1,2,3,4,5,6,7])
'''

"\ndef column_tensor_product(lhs: np.ndarray, rhs: np.ndarray) -> np.ndarray:\n    print('lhs col tensor', lhs)\n    print('rhs col tensor', rhs)\n    return np.tensordot(lhs, rhs, axes=0)              .reshape(len(lhs) * len(rhs))\n\ndef generate_state_tensor(states: list[np.ndarray]) -> np.ndarray:\n    return reduce(\n        lambda accum, curr: column_tensor_product(accum, curr),\n        states\n    )\n\ndef tensor_product_equations(flattened_state_vecs: list[float], feature_values: list[float]) -> list[float]:\n    state_vecs = np.array(flattened_state_vecs)         .reshape(int(len(flattened_state_vecs) / 2), 2)\n    print('TP equalities. state_vecs', state_vecs)\n    print('TP equalities. feature_values', feature_values)\n    state_tensor = generate_state_tensor(state_vecs)\n    \n    print('TP equalities. state_tensor', state_tensor)\n    # each component of the state tensor needs to be equal to that component from the feature vector\n    # and each qubit state vector must hav

## Training